 # Time series prediction
 
 Predict kW 48 hours ahead of current time.
 
 In this data reading of meter is bi-hourly. We have to get houly reading in predictions. 
 
 For optimization, our prediction shold not be deviated more than ±5kWh.
 
 ## To Do
 
 1- Sampling (convert bi-hourly data into hourly data)
 
 2- Filling the Data (entering the missing values) ✔️
 
 3- Sorting the data timewise (Status: done) ✔️
 
 4- Analysis
 
 How to make Time_stamp indexed? ✔️
 
 How to fill hourly missing value using pandas? ✔️
 
 ### Sub To do
 
 1-Fill hourly missing value. ✔️
 
 2-Then mean the day values. ✔️

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
#from plt import rcParams
#rcPramas['figure.figsize'] = 10, 6
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

In [ ]:
dataset = pd.read_csv("3098000032-Mar18-Mar19.csv")

In [ ]:
dataset.head(50)

In [ ]:
dataset.describe()

In [ ]:
dataset.shape

In [ ]:
dataset.dtypes

In [ ]:
#For checking missing values in dataset
dataset.isna().sum()

dataset.kW.plot()

In [ ]:
dataset['Time_stamp'] = pd.to_datetime(dataset['Time_stamp'], infer_datetime_format=True)
indexedDataset = dataset.set_index(['Time_stamp'])
df = pd.DataFrame(dataset, columns=['kW', 'Time_stamp'])

In [ ]:
#df['Time'] = pd.to_datetime(dataset.Time_stamp)

In [ ]:
import matplotlib.pyplot as plt
df.plot(x='Time_stamp', y='kW', kind='line') 
plt.show()

In [ ]:
#Number of time_stamp per month
df.Time_stamp.dt.month.value_counts()

In [ ]:
#Line plot of Number of time_stamp per month
df.Time_stamp.dt.month.value_counts().sort_index().plot()

In [ ]:
df.Time_stamp.min()

In [ ]:
df.Time_stamp.max()

In [ ]:
df.loc[200:250]

## Sorting w.r.t time
Status: Done

In [ ]:
df.set_index('Time_stamp', inplace=True)

In [ ]:
df.dtypes

In [ ]:
#Sorting Time_wise

df = df.sort_values(['Time_stamp'])

In [ ]:
df.shape

In [ ]:
df.head(31)

In [ ]:
df.loc['2018-11-12 02:00:00':]

In [ ]:
fd = df.loc['2018-11-01 15:30:00':]

In [ ]:
fd.plot()

In [ ]:
#Actual Data to work on.
df = df.loc['2018-03-05 09:00:00':'2018-11-01 15:30:00']
df.plot()

### Rolling mean & Rolling statistics

In [ ]:
#calculate rolling mean
rollmean = df.rolling(window=30).mean()

#Calculate standard deviation
rollstd = df.rolling(window=30).std()

print(rollmean, rollstd)

In [ ]:
origin = plt.plot(df, color='blue', label="Orignal")
mean = plt.plot(rollmean, color='red', label="Rolling Mean")
std = plt.plot(rollstd, color='black', label="Rolling std")
plt.legend(loc='best')
plt.title('Rolling Mean and Standard Devision')
plt.show(block=False)

In [ ]:
#Dickey-Fuller Test
From statsmodels.tsa.stattools as adfuller

print("Dickey fuller test result:")
dftest = adfuller(df['kW'], autolag='AIC')

dfoutput = pd.Series(df, index=['Test statistic', 'p-value', '#Lags Used', 'Number of Observations used'])
for key, value in dftest.items():
    dfoutput['Critical Value (&s)'%key] = value
    
print(dfoutput)

In [ ]:
fd = df

In [ ]:
fd


In [ ]:
fd.interpolate(method ='linear', limit_direction ='forward') 

In [ ]:
fd.fillna(method ='pad') 

In [ ]:
fd.fillna(fd.mean())

### Data preprocessing

In [ ]:
# Train test split
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

In [ ]:
#Create dataset function for LSTM model

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:


time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.kW, time_steps)
X_test, y_test = create_dataset(test, test.kW, time_steps)

print(X_train.shape, y_train.shape)

## LSTM Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from pylab import rcParams
from matplotlib import rc

%config InlineBackend.figure_format='retina'


sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

### Modeling

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(0.001))

### Training

In [ ]:
history = model.fit(
    X_train, y_train, 
    epochs=20, 
    batch_size=16, 
    validation_split=0.1, 
    verbose=1, 
    shuffle=False
)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train, 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test, marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
plt.plot(y_test, marker='.', label="true")
plt.plot(y_pred, 'r', label="prediction")
plt.ylabel('Value')
plt.xlabel('Time Step')
plt.legend()
plt.show();